<center>IF you find this notebook in anyway helpfull please 👍</center>

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from termcolor import colored
import plotly.express as px
import plotly.graph_objects as go

color = ["#EFF0EB", '#2D2D2F', '#0B5F6A', '#B0DCE7', '#60365E' ]


def divider():
    print('----------------------------------------------------------------------------------------------------------------------------\n')

In [ ]:
divider()
train_data = pd.read_csv('../input/spaceship-titanic/train.csv')
print (f'Importing train Data - Data Shape  -')
print(f'\n We have {train_data.shape[0]} rows and {train_data.shape[1]} columns -\n')
train_data = pd.concat([train_data, train_data.Cabin.str.split("/", expand=True).iloc[:, [0,1,-1]].rename(columns={0:'Deck', 1:'Cabin Number', 2:'Side'})], axis=1)
display(train_data.head(2))
divider()
test_data = pd.read_csv('../input/spaceship-titanic/test.csv')
print (f'Importing test Data - Data Shape  -')
print(f'\n We have {test_data.shape[0]} rows and {test_data.shape[1]} columns in Dataset')
test_data = pd.concat([test_data, test_data.Cabin.str.split("/", expand=True).iloc[:, [0,1,-1]].rename(columns={0:'Deck', 1:'Cabin Number', 2:'Side'})], axis=1)
display(test_data.head(2))

In [ ]:

divider()

print("Let's Explore features type:- \n")
#Dtype == Object
object_col = list(train_data.select_dtypes(include=['object', 'category']).columns)
print(f'Number of "Textual Features" in Train Set are {len(object_col)} :-   {", ".join(object_col)}')

#Dtype == int or float
num_col = list(train_data.select_dtypes(include=['int64', 'float']).columns)
print(f'\nNumber of "Numerical Features" in Train Set are {len(num_col)} :-  {", ".join(num_col)}')

In [ ]:
divider()

print("Let's check Null Value \n")

null_cols1 = train_data.columns[train_data.isnull().sum()!=0]
df1 = train_data.isnull().sum()[null_cols1].to_frame(name = 'Train_Null_Count')
df1['Train_Null_Percent %'] = round(df1.Train_Null_Count / train_data.shape[0] * 100, 2)

null_cols1 = test_data.columns[test_data.isnull().sum()!=0]
df2 = test_data.isnull().sum()[null_cols1].to_frame(name = 'Test_Null_Count')
df2['Test_Null_Percent %'] = round(df2.Test_Null_Count / test_data.shape[0] * 100, 2)

df1 = pd.concat([df1, df2], axis=1)
df1.sort_values('Train_Null_Count', inplace=True)
display(
    df1.style.highlight_max(color='#FF7F00').highlight_min(color='lightgreen')
)

<center> This gonna be exiting. Lots of Null value</center>

In [ ]:
divider()

def font_size(s, props=[]):
    color = [" " for i in range(len(s))]
    color[0] = props[-1]
    color[-1] = props[-1]
    color[3] = props[-1]
    if s[0]== 'PassengerId':
        for i in range(len(s)):
            if i == 2 or i == 4:
                color[i] = props[0]
                
    return color

Features = []
for i in train_data.columns:
    try:
        Features.append([i,train_data[i].dtype, train_data[i].nunique(), train_data[i].drop_duplicates().values,
                     int(test_data[i].nunique()), test_data[i].drop_duplicates().values])
    except:
        Features.append([i,train_data[i].dtype, train_data[i].nunique(), train_data[i].drop_duplicates().values])
display(pd.DataFrame(Features, columns = ['Features','Dtype', 'Train Unique Count', 'Train Unique Values', 'Test Unique Count', 'Test Unique value']).style
        .set_caption("Let's Check All Colums")
        .format(precision=0)
        .hide_index()
        .apply(font_size, props=["font-size: 18px; color:red; border: 2px solid red",
                                  """display: block;width: 300px;
                                      text-overflow: ellipsis;
                                      word-wrap: break-word;
                                      overflow: hidden;
                                      max-height: 1.8em;
                                      line-height: 1.8em;""" ], axis=1))
print(f"""
   ** There are {colored(8693, 'red')} Passengers in train and {colored(4277, 'red')} passengers in test 
""")

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Lets do some EDA</b></p>
</div>

In [ ]:
divider()

color = ["#EFF0EB", '#2D2D2F', '#0B5F6A', '#B0DCE7', '#60365E' ]


bg_color = color[0]
fig = plt.figure(figsize = (25,15), dpi=150)
fig.patch.set_facecolor(bg_color)
gs = fig.add_gridspec(4,3)
gs.update(wspace=0.35, hspace=0.27)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax8 = fig.add_subplot(gs[1,1])
ax4 = fig.add_subplot(gs[1,2])
ax5 = fig.add_subplot(gs[2,0])
ax6 = fig.add_subplot(gs[2,1])
ax7 = fig.add_subplot(gs[2,2])
ax9 = fig.add_subplot(gs[3,0])
ax10 = fig.add_subplot(gs[3,1])
ax11 = fig.add_subplot(gs[3,2])
ax0.text(-25.5, 0.04, 'Compairing            and          Dataset', fontsize=20, fontweight='bold', fontfamily='serif', color=color[1])
ax0.text(17, 0.04, 'Train', fontsize=25, fontweight='bold', fontfamily='serif', color=color[3])
ax0.text(57, 0.04, 'Test', fontsize=25, fontweight='bold', fontfamily='serif', color=color[4])

ax0.text(-20.5, 0.035, 'Age', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax0.grid(color=color[2], linestyle=":", axis="x",zorder=0,dashes=(1,10) )
sns.kdeplot(data=train_data, x='Age', ax=ax0, color=color[3], shade=True,  bw_adjust=2)
sns.kdeplot(data=test_data, x='Age', ax=ax0, color=color[4], shade=True, bw_adjust=2)
ax0.set_xlim([-19, 100])
ax0.set_ylabel(" ")
ax0.yaxis.set_major_locator(mtick.MultipleLocator(0.01))

ax1.text(-2000.5, 0.0017, 'RoomService', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax1.grid(color=color[2], linestyle=":", axis="x",zorder=0,dashes=(1,10) )
sns.kdeplot(data=train_data, x='RoomService', ax=ax1, color=color[3], shade=True,  bw_adjust=2)
sns.kdeplot(data=test_data, x='RoomService', ax=ax1, color=color[4], shade=True, bw_adjust=2)
ax1.set_ylabel(" ")
ax1.yaxis.set_major_locator(mtick.MultipleLocator(0.0005))

ax2.text(-4000.5, 0.00075, 'FoodCourt', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax2.grid(color=color[2], linestyle=":", axis="x",zorder=0,dashes=(1,10) )
sns.kdeplot(data=train_data, x='FoodCourt', ax=ax2, color=color[3], shade=True,  bw_adjust=2)
sns.kdeplot(data=test_data, x='FoodCourt', ax=ax2, color=color[4], shade=True, bw_adjust=2)
ax2.set_ylabel(" ")
ax2.yaxis.set_major_locator(mtick.MultipleLocator(0.0002))

ax3.text(-3000.5, 0.00185, 'ShoppingMall', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax3.grid(color=color[2], linestyle=":", axis="x",zorder=0,dashes=(1,10) )
sns.kdeplot(data=train_data, x='ShoppingMall', ax=ax3, color=color[3], shade=True,  bw_adjust=2)
sns.kdeplot(data=test_data, x='ShoppingMall', ax=ax3, color=color[4], shade=True, bw_adjust=2)
ax3.set_ylabel(" ")
ax3.yaxis.set_major_locator(mtick.MultipleLocator(0.0005))

ax = ax4
ax.text(-3000.5, 0.001, 'Spa', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="x",zorder=0,dashes=(1,10) )
sns.kdeplot(data=train_data, x='Spa', ax=ax, color=color[3], shade=True,  bw_adjust=2)
sns.kdeplot(data=test_data, x='Spa', ax=ax, color=color[4], shade=True, bw_adjust=2)
ax.set_ylabel(" ")
ax.yaxis.set_major_locator(mtick.MultipleLocator(0.0005))

ax = ax8
ax.text(-3000.5, 0.001, 'VRDeck', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="x",zorder=0,dashes=(1,10) )
sns.kdeplot(data=train_data, x='VRDeck', ax=ax, color=color[3], shade=True,  bw_adjust=2)
sns.kdeplot(data=test_data, x='VRDeck', ax=ax, color=color[4], shade=True, bw_adjust=2)
ax.set_ylabel(" ")
ax.yaxis.set_major_locator(mtick.MultipleLocator(0.0005))

data = train_data['HomePlanet'].value_counts()
data2 = test_data['HomePlanet'].value_counts()
ax = ax6
ax.text(-0.5, 5200, 'HomePlanet', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="y",zorder=0,dashes=(1,10) )
x = np.arange(3)
ax.bar(x, height=data, zorder=3, color=color[3], width=0.9, alpha=0.8)
ax.bar(x, height=data2,  zorder=3, color=color[4], width=0.5, alpha=0.5)
ax.set_ylabel(" ")
ax.set_xticks(x)
ax.set_xticklabels(labels = data.index.tolist())
ax.yaxis.set_major_locator(mtick.MultipleLocator(1000))


ax = ax7
col = 'CryoSleep'
data = train_data[col].value_counts()
data2 = test_data[col].value_counts()
ax.text(-0.5, 6000, col, fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="y",zorder=0,dashes=(1,10) )
x = np.arange(2)
ax.bar(x, height=data, zorder=3, color=color[3], width=0.7, alpha=0.8)
ax.bar(x, height=data2,  zorder=3, color=color[4], width=0.3, alpha=0.5)
ax.set_ylabel(" ")
ax.set_xticks(x)
ax.set_xticklabels(labels = data.index.tolist())
ax.yaxis.set_major_locator(mtick.MultipleLocator(1000))

ax = ax5
col = 'Destination'
data = train_data[col].value_counts()
data2 = test_data[col].value_counts()
ax.text(-0.5, 6700, col, fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="y",zorder=0,dashes=(1,10) )
x = np.arange(3)
ax.bar(x, height=data, zorder=3, color=color[3], width=0.9, alpha=0.8)
ax.bar(x, height=data2,  zorder=3, color=color[4], width=0.5, alpha=0.5)
ax.set_ylabel(" ")
ax.set_xticks(x)
ax.set_xticklabels(labels = data.index.tolist())
ax.yaxis.set_major_locator(mtick.MultipleLocator(1000))

ax = ax9
col = 'VIP'
data = train_data[col].value_counts()
data2 = test_data[col].value_counts()
ax.text(-0.5, 9000, col, fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="y",zorder=0,dashes=(1,10) )
x = np.arange(2)
ax.bar(x, height=data, zorder=3, color=color[3], width=0.9, alpha=0.8)
ax.bar(x, height=data2,  zorder=3, color=color[4], width=0.5, alpha=0.5)
ax.set_ylabel(" ")
ax.set_xticks(x)
ax.set_xticklabels(labels = data.index.tolist())
ax.yaxis.set_major_locator(mtick.MultipleLocator(3000))

ax = ax10
col = 'Deck'
data = train_data[col].value_counts()
data2 = test_data[col].value_counts()
ax.text(-0.5, 3100, col, fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="y",zorder=0,dashes=(1,10) )
x = np.arange(len(data))
ax.bar(x, height=data, zorder=3, color=color[3], width=0.9, alpha=0.8)
ax.bar(x, height=data2,  zorder=3, color=color[4], width=0.5, alpha=0.5)
ax.set_ylabel(" ")
ax.set_xticks(x)
ax.set_xticklabels(labels = data.index.tolist())
ax.yaxis.set_major_locator(mtick.MultipleLocator(1000))

ax = ax11
col = 'Side'
data = train_data[col].value_counts()
data2 = test_data[col].value_counts()
ax.text(-0.5, 4600, col, fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ax.grid(color=color[2], linestyle=":", axis="y",zorder=0,dashes=(1,10) )
x = np.arange(len(data))
ax.bar(x, height=data, zorder=3, color=color[3], width=0.7, alpha=0.8)
ax.bar(x, height=data2,  zorder=3, color=color[4], width=0.3, alpha=0.5)
ax.set_ylabel(" ")
ax.set_xticks(x)
ax.set_xticklabels(labels = data.index.tolist())
ax.yaxis.set_major_locator(mtick.MultipleLocator(1000))

<center>**Nothing ODD found here**</center>

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Lets check where spaceship heading?</b></p>
</div>
Means where people are and where they want to go

In [ ]:
divider()
df = pd.concat([train_data, test_data]).groupby(['HomePlanet', 'Destination']).size()
df = pd.concat([df, df.groupby(level=0).apply(lambda x: (x/x.sum()*100).round(1))], axis=1).reset_index()
df = df.rename(columns ={0:'count', 1:'percentage'} )


fig = px.sunburst(df, path=['HomePlanet', 'Destination'],
                  values='count', color='HomePlanet',color_discrete_sequence=[color[2], color[3], color[4]],
                  title='Spaceship "Titanic" final Destination', width=1000, height=500,
                  custom_data=['percentage']
                  
                 ) 
fig.update_traces(hovertemplate="<br>".join([
    "Count:%{value}",
]),
                  textinfo="label+percent parent"
)

fig.update_layout(hoverlabel=dict(
    bgcolor=color[1],
    font_size=16,
    font_color=color[0],
    font_family="serif"
),
                  paper_bgcolor=color[0],

                 
)
fig.add_annotation(
        x=0.95, y=0.99, xref="paper", yref="paper",
        text=f"""
        * Total Passengers are {df['count'].sum()}<br>
        * 54% passengers are from Earth.<br>
        * Out of which, 70% are heading <br>
          to TRAPPIST-1e
        """,
        showarrow=False,font=dict(family="serif", size=16, color=color[1]),
        align="left"
        )
fig.add_annotation(
        x=0, y=-0.250, xref="paper", yref="paper",
        text=f"""
        * Total Passenger are 12970 but due to null vlaues it's cut down to {df['count'].sum()}<br>
        """,
        showarrow=False,font=dict(family="serif", size=12, color=color[1]),
        align="left"
        )
fig.show()

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Let check the "Titanic" Spaceship</b></p>
</div>

In [ ]:
df = pd.concat([train_data, test_data])
ch1 = df.groupby(['Deck', 'Side']).agg({'Cabin Number':['count', 'nunique', 'unique']})
ch1.reset_index(inplace=True)


def check(x):
    info = list()
    uni = x['Cabin Number']['nunique']
    li_c_n = x['Cabin Number']['unique']
    li_c_n = [int(i) for i in li_c_n]
    total_cabin_per_deck = max(li_c_n)+1
    info.append(total_cabin_per_deck)
    lis = np.arange(total_cabin_per_deck)
    not_in_cabin_number = [i for i in lis if i not in li_c_n]
    info.append(len(not_in_cabin_number))
    return info

ch2 = ch1.apply(lambda x: check(x), axis=1)

ch1['data'] = ch2

ch1.loc[:, 'Max_Cabin'] = ch1.data.apply(lambda x:x[0])
ch1.loc[:, 'Empty_Cabin'] = ch1.data.apply(lambda x:x[-1])

side = ch1.groupby('Side').agg({('Max_Cabin', ''):'sum', ('Empty_Cabin', ''):'sum'})
deck = ch1.groupby('Deck').agg({('Max_Cabin', ''):'sum', ('Empty_Cabin', ''):'sum'})

fig = go.Figure()


fig.update_layout(
    paper_bgcolor=color[0],
    plot_bgcolor=color[0],
    shapes = [
        dict(
            type="path",
            path=" M 2,4 L1,6 L6,6 L5,4, Z",
            fillcolor=color[4],
            opacity=0.5,
            line_color=color[1],
        ),
        dict(
            type="path",
            path=" M 2,8 L1,6 L6,6 L5,8, Z",
            fillcolor=color[2],
            opacity = 0.5,
            line_color=color[1],
        ),
        
    ],
    
    xaxis = dict(
        range=[0-4, 8+4],
        zeroline=False,
        showgrid=False,
        showticklabels=False
    ),
    yaxis = dict(
        range=[-10, 15],
        zeroline=False,
        showgrid=False,
        showticklabels=False
    ),
    width=1000, height=500
)
fig.add_annotation(
        x=0.5, y=1.09, xref="paper", yref="paper",
        text=f"""
        <Span style="color:{color[2]};font-size:20px;">Port Side</Span><br>
        * Total Cabins 5017<br>
        * Cabins in all 8 decks
        """,
        showarrow=False,font=dict(family="serif", size=16, color=color[1]),align="left")
fig.add_annotation(
        x=0.5, y=0.19, xref="paper", yref="paper",
        text=f"""
        <Span style="color:{color[4]};font-size:20px;">Starboard Side</Span><br>
        * Total Cabins 4997<br>
        * Cabins in all 8 decks
        """,
        showarrow=False,font=dict(family="serif", size=16, color=color[1]),align="left")
fig.add_annotation(
        x=-0.05, y=0.59, xref="paper", yref="paper",
        text=f"""
        <Span style="color:{color[1]};font-size:30px;">"Titanic" Spaceship</Span><br>
        * Total Cabins 10014<br>
        
        """,
        showarrow=False,font=dict(family="serif", size=20, color=color[1]),align="left")
fig.add_annotation(
        x=0.84, y=1.09, xref="paper", yref="paper",
        text=f"""
        <Span style="color:{color[1]};font-size:20px;">Decks</Span><br>
        * Total 8 decks<br>
        
        """,
        showarrow=False,font=dict(family="serif", size=20, color=color[1]),align="left")
fig.add_annotation(
        x=0.9, y=0.89, xref="paper", yref="paper",
        text=f"""
        * Deck A - 209 Cabin<br>
        * Deck B - 656 Cabin<br>
        * Deck C - 653 Cabin<br>
        * Deck D - 577 Cabin<br>
        * Deck E - 1208 Cabin<br>
        * Deck F - 3692 Cabin<br>
        * Deck G - 3010 Cabin<br>
        * Deck T - 9 Cabin
        """,
        showarrow=False,font=dict(family="serif", size=17, color=color[1]),align="left")
fig.add_annotation(
        x=0, y=-0.250, xref="paper", yref="paper",
        text=f"""
        <Span style="color:{color[1]};font-size:15px;">Assumption</Span><br>
        Cabin Number in both Port & Starboard side and in the different deck starts with 0 and end with max Cabin number available in a particular group <br>
        """,
        showarrow=False,font=dict(family="serif", size=12, color=color[1]),
        align="left"
        )

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Let's check About passengers</b></p>
</div>

In [ ]:
df = pd.concat([train_data, test_data]).reset_index(drop=True)

d1 = df.HomePlanet.value_counts()
earth_p = d1['Earth']
europa_p = d1['Europa']
mars_p = d1['Mars']
d1 = df.query('VIP == True').HomePlanet.value_counts()
vip_europa = d1['Europa']
vip_mars = d1['Mars']

color = ["#EFF0EB", '#2D2D2F', '#0B5F6A', '#B0DCE7', '#60365E' ]
cearth = color[2]
cmars = color[4]
ceuropa = color[1]
bg_color = color[0]
fig = plt.figure(figsize = (20,15), dpi=150)
fig.patch.set_facecolor(bg_color)
gs = fig.add_gridspec(4,2)
gs.update(wspace=0.35, hspace=0.47)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])
ax4 = fig.add_subplot(gs[2,0])
ax5 = fig.add_subplot(gs[2,1])
ax6 = fig.add_subplot(gs[3,0])


ax=ax0
ax.text(-10, 0.035, 'Passengers', fontsize=25, fontweight='bold', fontfamily='serif', color=color[1])
ax.text(-10, 0.029, 'Age Distribution', fontsize=20, fontweight='bold', fontfamily='serif', color=color[1])
sns.kdeplot(data=df,ax=ax, x='Age', hue='HomePlanet', shade=True, alpha=0.5, palette=[ceuropa, cearth, cmars])
sns.move_legend(ax, "upper center", bbox_to_anchor=(0.3, 1.13), ncol=3, title=None, frameon=False)
ax.annotate(f'Europa - {europa_p:0.0f}', xy=(60, 0.02), color=ceuropa,fontsize=15,
                 fontfamily='serif', horizontalalignment='left', verticalalignment='center')
ax.annotate(f'Earth - {earth_p:0.0f}', xy=(60, 0.017), color=cearth,fontsize=15,
                 fontfamily='serif', horizontalalignment='left', verticalalignment='center')
ax.annotate(f'Mars - {mars_p:0.0f}', xy=(60, 0.014), color=cmars,fontsize=15,
                 fontfamily='serif', horizontalalignment='left', verticalalignment='center')
ax.set_ylabel(" ")
ax.set_yticks([])

ax= ax1
ax.text(5, 0.039, 'VIPs Age Distribution', fontsize=20, fontweight='bold', fontfamily='serif', color=color[1])
sns.kdeplot(ax=ax,data=df.query('VIP == True'), x='Age', hue='HomePlanet', shade=True, alpha=0.5, palette=[ceuropa, cmars])
ax.legend([],[], frameon=False)
ax.annotate(f'Europa - {vip_europa:0.0f}', xy=(60, 0.027), color=ceuropa,fontsize=15,
                 fontfamily='serif', horizontalalignment='left', verticalalignment='center')
ax.annotate(f'Mars - {vip_mars:0.0f}', xy=(60, 0.023), color=cmars,fontsize=15,
                 fontfamily='serif', horizontalalignment='left', verticalalignment='center')
ax.set_ylabel(" ")
ax.set_yticks([])



for y_val, ax in zip(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], [ax2, ax3, ax4, ax5, ax6]):
    sns.violinplot(ax=ax,data=df, x='HomePlanet', y=y_val, palette=[ceuropa, cearth, cmars], alpha=0.5)
    ax.yaxis.set_major_locator(mtick.MultipleLocator(10000))
    ax.set_ylim([-5000,32000])
ax2.text(-0.35, 35000, 'Billing Distribution on various luxury amenities', fontsize=20, fontweight='bold', fontfamily='serif', color=color[1])

fig.text(0.55, 0.27, 'Insight', fontsize=20, fontweight='bold', fontfamily='serif', color='#323232')
fig.text(0.55, 0.12, """
*  260 Passenger's age is lower than 1.
*  54% of Passengers are from Earth.
*  No VIP are from earth, mean in future who have money,
    fame and power will leave earth for better place
*  In Europa, passenger age lower than 10 are lowest
    and vip highest
*  Europa people love spending on luxury amenities,
    especially on foodcourt
""",fontsize=15, fontweight='bold', fontfamily='serif', color='#323232')

plt.show()

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-famil`my:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Let's Dig deep in Cabin, ie. Passenger per cabin</b></p>
</div>

In [ ]:
df = pd.concat([train_data, test_data]).reset_index(drop=True)

df['passenger_per_cabin'] = df.groupby('Cabin')['Deck'].transform('count')
df['passenger_per_cabin'].fillna(0, inplace=True)
df['passenger_per_cabin'] = df['passenger_per_cabin'].astype('int')

bg_color = color[0]
fig = plt.figure(figsize = (20,15), dpi=150)
fig.patch.set_facecolor(bg_color)
gs = fig.add_gridspec(4,2)
gs.update(wspace=0.35, hspace=0.47)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])
ax4 = fig.add_subplot(gs[2,0])

ax = ax0
ax.text(0, 9500, 'Passenger per Cabin', fontsize=20, fontweight='bold', fontfamily='serif', color=color[1])
ch1 = df[df['passenger_per_cabin']>0]
sns.countplot(x='passenger_per_cabin', ax=ax, data=ch1, palette=[color[3]])
for bar in ax.patches:
    if bar.get_x() < 0:
        bar.set_color(color[4])  
ax.set_xlabel("")
ax.set_ylabel("")

ax=ax1
ax.text(0, 1300, 'Passenger per Cabin = 1', fontsize=18, fontweight='bold', fontfamily='serif', color=color[1])
ch1 = df.query('passenger_per_cabin == 1')
sns.histplot(data=ch1, x='Age', bins=8,hue='Transported', palette = [color[3], color[4]], ax=ax)
ax.set_xlabel('Age Distribution')
ax.set_ylabel("")

ax=ax2
ax.text(0, 270, 'Passenger per Cabin = 2', fontsize=18, fontweight='bold', fontfamily='serif', color=color[1])
ch1 = df.query('passenger_per_cabin == 2')
sns.histplot(data=ch1, x='Age', bins=8,hue='Transported', palette = [color[3], color[4]], ax=ax)
ax.set_xlabel('Age Distribution')
ax.set_ylabel("")

ax=ax3
ax.text(0, 165, 'Passenger per Cabin = 3', fontsize=18, fontweight='bold', fontfamily='serif', color=color[1])
ch1 = df.query('passenger_per_cabin == 3')
sns.histplot(data=ch1, x='Age', bins=8,hue='Transported', palette = [color[3], color[4]], ax=ax)
ax.set_xlabel('Age Distribution')
ax.set_ylabel("")

ax=ax4
ax.text(0, 185, 'Passenger per Cabin = 4 or more', fontsize=18, fontweight='bold', fontfamily='serif', color=color[1])
ch1 = df.query('passenger_per_cabin >= 4')
sns.histplot(data=ch1, x='Age', bins=8,hue='Transported', palette = [color[3], color[4]], ax=ax)
ax.set_xlabel('Age Distribution')
ax.set_ylabel("")

fig.text(0.55, 0.47, 'Insight', fontsize=20, fontweight='bold', fontfamily='serif', color='#323232')
fig.text(0.55, 0.37, """
*  8141 are travelling in heir personal cabin.
*  Out of 8141, 162 passenger are of age <= 10.(Strange!!)
*  As the Passenger per cabin increases, chance of surviving 
   also increases.
""",fontsize=15, fontweight='bold', fontfamily='serif', color='#323232')
plt.show()

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-famil`my:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Let's check what's this CryoSleep is?</b></p>
</div>

In [ ]:
df = pd.concat([train_data, test_data]).reset_index(drop=True)
bg_color = color[0]
fig = plt.figure(figsize = (20,15), dpi=150)
fig.patch.set_facecolor(bg_color)
gs = fig.add_gridspec(4,2)
gs.update(wspace=0.35, hspace=0.47)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])

fig.text(0.1, 0.95, 'CryoSleep -', fontsize=30, fontweight='bold', fontfamily='serif', color=color[2])
fig.text(0.25, 0.945, 'Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage.\nPassengers in cryosleep are confined to their cabins.', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])


ax = ax0
ax.text(-0.5, 9000, 'Distribution', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
df.CryoSleep.value_counts().plot(kind='bar', ax=ax, color=[color[1], color[2]])

ax=ax1
ax.text(-3, 800, 'Age Distribution given CryoSleep =  True', fontsize=15, fontweight='bold', fontfamily='serif', color=color[2])
ch1 = df.query('CryoSleep == True')
sns.histplot(data=ch1, x='Age', bins=8,hue='Transported', palette = [color[3], color[4]], ax=ax)

ax=ax2
ax.text(-3, 1400, 'Age Distribution given CryoSleep =  False', fontsize=15, fontweight='bold', fontfamily='serif', color=color[1])
ch1 = df.query('CryoSleep == False')
sns.histplot(data=ch1, x='Age', bins=8,hue='Transported', palette = [color[3], color[4]])

fig.text(0.55, 0.67, 'Insight', fontsize=20, fontweight='bold', fontfamily='serif', color='#323232')
fig.text(0.55, 0.58, """
*  4581 paasengers in CryoSleep.
*  If you are in CryoSleep, chances to get transported
   increases.
""",fontsize=15, fontweight='bold', fontfamily='serif', color='#323232')
plt.show()

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-famil`my:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Let's check Passenger's Relation, if any!!</b></p>
</div>

In [ ]:
df = pd.concat([train_data, test_data]).reset_index(drop=True)
df['p_code'] = df.PassengerId.str.split("_", expand=True)[0]
df['group_of_people'] = df.groupby('p_code')['PassengerId'].transform('count')
ch1 = df.group_of_people.value_counts().reset_index()
ch1.columns = ['Passenger_together', 'Sum_of_Passenger']
ch1['Family_count'] = (ch1.Sum_of_Passenger/ch1.Passenger_together).astype('int')
ch1.sort_values('Passenger_together')

7145 Solo travellers!! nice

In [ ]:
fig = plt.figure(figsize = (20,3), dpi=150)
fig.patch.set_facecolor(bg_color)
gs = fig.add_gridspec(1,2)
gs.update(wspace=0.15, hspace=0.37)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
# ax2 = fig.add_subplot(gs[1,0])

ax = ax0
ax.text(0, 9500, 'Passengers Info', fontsize=20, fontweight='bold', fontfamily='serif', color=color[1])
# ch1 = df[df['passenger_per_cabin']>0]
sns.barplot(y='Sum_of_Passenger',x='Passenger_together', ax=ax, data=ch1, palette=[color[3]])
for bar in ax.patches:
    if bar.get_x() < 0:
        bar.set_color(color[4])  
ax.set_xlabel("Passenger travelling with")
ax.set_ylabel("")
for i, j, k in zip([0,1,2,3], ch1['Sum_of_Passenger'], ch1['Sum_of_Passenger'],):
    ax0.annotate(f'{k:0.0f}', xy=(i, j/2), color=color[0],fontsize=10, horizontalalignment='center', verticalalignment='center')
ax.annotate(f'7145 are solo travellers, 2590 are family of two', xy=(1, 6000), color=color[4],fontsize=15,
                 fontfamily='serif', horizontalalignment='left', verticalalignment='center')

ax = ax1
ax.text(0, 3000, 'Solo Traveller Age Distribution', fontsize=16, fontweight='bold', fontfamily='serif', color=color[1])
sns.histplot(data=df.query('group_of_people == 1'),ax=ax,bins=8, x='Age', alpha=0.5, color=[color[3]])
ax.annotate(f'84 are below 10 &\n32 are above 70', xy=(40, 2000), color=color[4],fontsize=15,
                 fontfamily='serif', horizontalalignment='left', verticalalignment='center')

plt.show()

<div style="color:white;display:fill;border-radius:8px;
    background-color:#323232;font-size:150%;
    font-famil`my:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>Handling Null Value</b></p>
</div>

In [ ]:
df = pd.concat([train_data, test_data]).reset_index(drop=True)
df['p_code'] = df.PassengerId.str.split("_", expand=True)[0]
df['group_of_people'] = df.groupby('p_code')['PassengerId'].transform('count')

df['passenger_per_cabin'] = df.groupby('Cabin')['Deck'].transform('count')
df['passenger_per_cabin'].fillna(0, inplace=True)
df['passenger_per_cabin'] = df['passenger_per_cabin'].astype('int')

df.head()

**1. HomePlanet**<br>
passenger who have same starting 4 digit must be from same planet.

In [ ]:
ch1 = df[['p_code', 'HomePlanet']].dropna().drop_duplicates()
df = df.merge(ch1, on='p_code', how='left')
df = df.rename(columns={'HomePlanet_x': 'HomePlanet', 'HomePlanet_y':'HomePlanet_v1'})
display(df.head(1))
print('\nremaining null values are - ', df.HomePlanet_v1.isnull().sum())

Person having same name or sirname must be of same planet

In [ ]:
ch1 = df[['Name', 'HomePlanet']]
ch2 = ch1.Name.str.split(" ", expand=True)
ch3 = pd.concat([ch2[0], ch2[1]])
ch4 = pd.concat([df.HomePlanet, df.HomePlanet])
ch5 = pd.concat([ch3, ch4], axis=1).reset_index(drop=True)
ch5.columns = ['Name', 'HomePlanet']
ch5.dropna(inplace=True)
ch5.drop_duplicates(inplace=True)
display(ch5.head(2))
ch6 = ch5.groupby(['Name'])['HomePlanet'].nunique().reset_index()
display(ch6.head(2))

In [ ]:
ch7 = df[df.HomePlanet_v1.isnull()]
ch8 = ch7.Name.str.split(" ", expand=True).reset_index()
ch9 = ch8.merge(ch6, left_on=0, right_on='Name', how='left').merge(ch5,left_on=0, right_on='Name', how='left').merge(ch6,left_on=1, right_on='Name', how='left').merge(ch5,left_on=1, right_on='Name', how='left')
ch9.fillna(0, inplace=True)
ch9['final_homeplanet'] = ch9.apply(lambda x: x.iloc[6] if x.iloc[6] != 0 else x.iloc[10], axis=1)
display(ch9.head(2))

In [ ]:
df.loc[ch9['index'], 'HomePlanet_v1'] = ch9.final_homeplanet.tolist()
print('\nremaining null values are - ', len(df[df.HomePlanet_v1==0]))

In [ ]:
df[df.HomePlanet_v1==0]

In same deck, passenger must be of same planet

In [ ]:
df.groupby(['Deck'])['HomePlanet'].unique()#.value_counts()

On G Deck there are only Earth Passenger

In [ ]:
df.loc[1855, 'HomePlanet_v1'] = 'Earth'

In [ ]:
df.groupby(['Deck', 'HomePlanet']).size()#.value_counts()

Deck which have highest HomePlanet number will take that is left 5. so for D Mars and For F Earth 

In [ ]:
df.loc[df[df.HomePlanet_v1==0].index, 'HomePlanet_v1'] = ['Mars', 'Earth', 'Earth', 'Earth', 'Earth']

So final count of HomePlanet is

In [ ]:
df.HomePlanet_v1.value_counts(dropna=False)

**2. Next handling CryoSleep**

In [ ]:
df.CryoSleep.value_counts(dropna=False)

In [ ]:
ch2 = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
ch1 = df.copy()
ch1[ch1.CryoSleep == True].loc[:, ch2].sum()

So passenger in CryoSleep dont spend on anything, make sense

If any spend by passenger then cryosleep is False

In [ ]:
ch1 = df[df.loc[:, ch2].sum(axis=1) != 0]
df.loc[ch1[ch1.CryoSleep.isna()].index, 'CryoSleep'] = False

In [ ]:
df.CryoSleep.value_counts(dropna=False)

Question is how to tackle these 136 left``

In [ ]:
# ch1 = df.groupby(['group_of_people', 'passenger_per_cabin', 'CryoSleep'], dropna=False).size().reset_index()
# ch1.query('group_of_people == 2 and passenger_per_cabin == 1')

In [ ]:
# sns.kdeplot(data=df, x='Age', shade=True, hue='CryoSleep')

In [ ]:
# df.groupby(['VIP', 'CryoSleep'], dropna=False).size()

In [ ]:
# sns.violinplot(data=df, x='CryoSleep', y='Spa', palette=[ceuropa, cearth, cmars], alpha=0.5)
# plt.show()

More to come!!!